## INTRODUCTION <a name="INTRODUCTION"></a>

In this project we will be analyzing the neighborhoods of Mumbai to find the location of already existing malls to determine the best location to open a new mall.

Note: Complete introduction will be in the final report pdf.

## Data <a name="data"></a>

Data Needed 

1. List of neighborhoods in Mumbai 

2. Lattitude and Longitude coordinates 

3. Data of venues 


Data Collection 

1. List of neighborhoods in Mumbai data via Wikipedia defines the scope of this project which is confined to the city of Mumbai. 

2. Python Geocoder package for latitude and longitude coordinates of the neighborhoods in order to plot the map and also to get the venue data. 

3. Foursquare API to get the venue data for the neighborhoods. 

4. Python Folium Package for Map Visualization. 

## CODE <a name="CODE"></a>

1. Importing Libraries needed 

In [6]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
!pip install BeautifulSoup4
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Libraries Imported")

     |████████████████████████████████| 122kB 5.3MB/s eta 0:00:01
Libraries Imported


In [7]:
!pip install folium # for map creation
import folium

2. Data Scrapping 

In [8]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai').text

In [9]:
soup = BeautifulSoup(data, 'html.parser') # parcing data html to to object

In [10]:
n = [] # list to store neighborhood data

In [11]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    n.append(row.text)

In [13]:
mum_df = pd.DataFrame({"Neighborhood": n})

mum_df.head() #data frame created of mumbai

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [14]:
mum_df.shape # no. of entries

(42, 1)

3.Getting Co-ordinates

In [15]:
def get_latlng(neighborhood):
    
    lat_lng_coords = None
   
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [17]:
cords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [18]:
cords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [20]:
df_cords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [21]:

# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [22]:
print(mum_df.shape)
mum_df #merged

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [23]:
# save the DataFrame as CSV file
mum_df.to_csv("mum_df.csv", index=False)

4. Creating map 

In [24]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [25]:
# creating map of Mumbai 
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [27]:

# save the map as HTML file
map_mum.save('map_mum.html')


5. Using foursquare API to explore the neighborhoods of mumbai

In [28]:
# Definw foursquare id
CLIENT_ID = 'UXHG10NZ2O5IOAXFCROC2MBHPJRCEI2HYXG1WIX0HRCAA4R4' 
CLIENT_SECRET = 'DACP4JYOSZH3BGTHY3YT5N0LTGE4XOW4ANNZB3MXKDTZPGVW' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UXHG10NZ2O5IOAXFCROC2MBHPJRCEI2HYXG1WIX0HRCAA4R4
CLIENT_SECRET:DACP4JYOSZH3BGTHY3YT5N0LTGE4XOW4ANNZB3MXKDTZPGVW


Finding the top 100 venues in 2000 m radius

In [30]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
   # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # getting only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2724, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [32]:
# checking the number of venues in each neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,17,17,17,17,17,17
Baiganwadi,6,6,6,6,6,6
Bandra,100,100,100,100,100,100
Bhandup,26,26,26,26,26,26
Borivali,100,100,100,100,100,100
Charkop,51,51,51,51,51,51
Chembur,100,100,100,100,100,100
Dahisar,56,56,56,56,56,56


In [33]:
# printing unique responses
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 191 uniques categories.


In [34]:
# showing the unique categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Falafel Restaurant',
       'Ice Cream Shop', 'Pub', 'Sandwich Place', 'Pizza Place',
       'Snack Place', 'Multiplex', 'Juice Bar', 'Seafood Restaurant',
       'Breakfast Spot', 'Coffee Shop', 'Café', 'Cocktail Bar',
       'American Restaurant', 'Department Store', 'Bar', 'Diner',
       'Chinese Restaurant', 'Maharashtrian Restaurant', 'BBQ Joint',
       'Lounge', 'Fast Food Restaurant', 'Gym / Fitness Center', 'Spa',
       'Residential Building (Apartment / Condo)', 'Asian Restaurant',
       'Electronics Store', "Women's Store", 'Smoke Shop',
       'Vegetarian / Vegan Restaurant', 'Food Truck', 'Liquor Store',
       'Athletics & Sports', 'Park', 'Tea Room', 'Martial Arts Dojo',
       'Fish Market', 'Food', 'Bus Station', 'Plaza', 'Sports Bar',
       'Music Venue', 'Hot Dog Joint', 'Gym', 'Dessert Shop',
       'Gourmet Shop', 'Sports Club', 'Restaurant'], dtype=object)

In [35]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

6. Analyzing the neighborhoods

In [36]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(2724, 192)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

 Grouping neighborhoods based on frequency of category

In [38]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped


(42, 192)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0.000000,0.0000,0.010000,0.000000,0.000000,0.020000,0.010000,0.010000,0.020000,0.0000,0.060000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.050000,0.00,0.050000,0.000000,0.03,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.0000,0.010000,0.00,0.000000,0.020000,0.00,0.010000,0.0000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.150000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.010000,0.020000,0.010000,0.000000,0.01,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.050000,0.0000,0.00,0.010000,0.000000,0.000000,0.00,0.000000,

In [39]:
len(mum_grouped[mum_grouped["Shopping Mall"] > 0]) #shopping malls 

19

Creating a dataframe for shopping malls only

In [40]:
mum_mall = mum_grouped[["Neighborhoods","Shopping Mall"]]

In [41]:
mum_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.038462


Using k - means to cluster neighborhoods 

In [42]:
# deviding into 3 clusters 
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 2, 0, 2, 0, 2], dtype=int32)

In [43]:
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [44]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.000000,0
4,Bhandup,0.038462,1


In [45]:
# merge Mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() 

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062940,72.926630
3,Bandra,0.000000,0,19.054370,72.840170
4,Bhandup,0.038462,1,19.145560,72.948560


In [46]:
# sorting results by labels of cluster
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.000000,0,18.996160,72.852790
32,"Sion, Mumbai",0.000000,0,19.043410,72.863320
31,Shil Phata,0.000000,0,19.146580,73.040050
30,Seven Bungalows,0.000000,0,19.131342,72.816342
28,Mumbra,0.000000,0,19.167632,73.021408
26,Mogra Village,0.000000,0,24.375900,75.954570
24,"Matharpacady, Mumbai",0.000000,0,18.950694,72.827268
23,Mankhurd,0.000000,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.000000,0,19.210940,72.841370


Showing clusters on map

In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

8 . Examining the clusters

In [49]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0] # red cluster ie 0 

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
35,Tilak Nagar (Mumbai),0.0,0,18.996160,72.852790
32,"Sion, Mumbai",0.0,0,19.043410,72.863320
31,Shil Phata,0.0,0,19.146580,73.040050
30,Seven Bungalows,0.0,0,19.131342,72.816342
28,Mumbra,0.0,0,19.167632,73.021408
26,Mogra Village,0.0,0,24.375900,75.954570
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
23,Mankhurd,0.0,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.0,0,19.210940,72.841370


In [54]:
# purple cluster ie 1
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Uttan,0.050000,1,26.866340,80.938840
4,Bhandup,0.038462,1,19.145560,72.948560
27,Mulund,0.037500,1,19.171830,72.955650
34,Thakur village,0.050000,1,19.210200,72.875410
37,Vashi,0.030000,1,19.084650,72.904810
10,Dombivli,0.043478,1,19.212750,73.083240
38,Vikhroli,0.030000,1,19.111090,72.927810
19,Kanjurmarg,0.064516,1,19.131380,72.935680
25,Mira Road,0.040000,1,19.265674,72.870681


In [55]:
# green cluster ie 2
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
39,Wadala,0.020202,2,19.017200,72.858170
13,Goregaon,0.020000,2,19.164550,72.849460
5,Borivali,0.010000,2,19.229360,72.857510
29,Pestom sagar,0.012346,2,19.070640,72.902170
7,Chembur,0.010000,2,19.062180,72.902410
9,Devipada,0.011111,2,19.224690,72.866050
40,Western Suburbs (Mumbai),0.012658,2,19.197010,72.827680
12,Ghatkopar,0.012500,2,19.086523,72.909008
33,"Sonapur, Bhandup",0.022222,2,19.163940,72.935440
41,Worli,0.020000,2,19.007440,72.816880


## Results and Discussion <a name="results"></a>

1. Cluster 0 has  zero to negligible presence of shopping malls in the neighborhoods.
1. Cluster 1 has the highest number of malls.
3. cluster 2 has moderate number of malls.


## Conclusion <a name="conclusion"></a>

A. Hence cluster 1 will is overpopulated in terms of malls which causes intense competition.
Developers are recommended to avoid these areas. These areas include mainly the western and a part of southern parts of Mumbai.


B. Cluster 2 is not over populated but the presence of competition will be noticed. Developers with new and innovative projects can be recommended this cluster to deal with the moderate competition.

C. Cluster 0 has negligible competition due to lack of malls. hence developers are strongly recommended to take advantages of these zones. These are the eastern and centrals parts of Mumbais.

In [ ]:
* Note: Results and conclusions will be discussed in da